# CS145: Project 3 | E-commerce Dataset with Biq Query

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* Arslan Ali, 2018-CE-236
* Muhammad Bilal, 2018-CE-247

**Setting Up BigQuery and Dependencies**

In [20]:
# Run this cell to authenticate yourself to BigQuery
from google.colab import auth
auth.authenticate_user()
project_id = "db-prj-ml"

In [21]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [22]:
# Add imports for any visualization libraries you may need
import matplotlib.pyplot as plt

%matplotlib inline

## Project Overview

1- Description:
The data analyst team at your Company exported the Google Analytics logs for an ecommerce website into BigQuery and created a new table of all the raw ecommerce visitor session data for me to explore. Using this data, I’ll try to answer a few questions.

Question1: Out of the total visitors who visited our website, what % made a purchase?

Question2: How many visitors bought on subsequent visits to the website?

Question3: What are some of the reasons a typical ecommerce customer will browse but not buy until a later visit?

Question4: Create a Machine Learning model in BigQuery to predict whether or not a new user is likely to purchase in the future. Identifying these high-value users can help the marketing team target them with special promotions and ad campaigns to ensure a conversion while they comparison shop between visits to your ecommerce site.


## Analysis of Dataset

---

*TODO: Analysis of your dataset*



----

## Data Exploration

---

*TODO: Exploring your questions, with appropriate visualizations*

---

##Question 1


In [23]:
%%bigquery --project $project_id
WITH visitors AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_visitors
FROM `data-to-insights.ecommerce.web_analytics`
),

purchasers AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_purchasers
FROM `data-to-insights.ecommerce.web_analytics`
WHERE totals.transactions IS NOT NULL
)

SELECT
  total_visitors,
  total_purchasers,
  total_purchasers / total_visitors AS conversion_rate
FROM visitors, purchasers 

,total_visitors,total_purchasers,conversion_rate
0,741721,20015,0.026985


##Question 2

In [24]:
%%bigquery --project $project_id
SELECT
  p.v2ProductName,
  p.v2ProductCategory,
  SUM(p.productQuantity) AS units_sold,
  ROUND(SUM(p.localProductRevenue/1000000),2) AS revenue
FROM `data-to-insights.ecommerce.web_analytics`,
UNNEST(hits) AS h,
UNNEST(h.product) AS p
GROUP BY 1, 2
ORDER BY revenue DESC
LIMIT 5;

,v2ProductName,v2ProductCategory,units_sold,revenue
0,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,17651,870976.95
1,Nest® Cam Outdoor Security Camera - USA,Nest-USA,16930,684034.55
2,Nest® Cam Indoor Security Camera - USA,Nest-USA,14155,548104.47
3,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,6394,178937.60
4,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,6340,178572.40


##Q3:

### visitors who bought on a return visit could have bought on first.

In [25]:
%%bigquery --project $project_id
WITH all_visitor_stats AS (
SELECT
  fullvisitorid, # 741,721 unique visitors
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)

SELECT
  COUNT(DISTINCT fullvisitorid) AS total_visitors,
  will_buy_on_return_visit
FROM all_visitor_stats
GROUP BY will_buy_on_return_visit 

,total_visitors,will_buy_on_return_visit
0,729848,0
1,11873,1


Analyzing the results, we can see that (11873 / 741721) = 1.6% of total visitors will return and purchase from the website. This includes the subset of visitors who bought on their very first session and then came back and bought again.

In [26]:
%%bigquery --project $project_id

select
  COUNT(DISTINCT fullVisitorId) AS unique_visitors,
  channelGrouping
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY channelGrouping
ORDER BY channelGrouping DESC;

,unique_visitors,channelGrouping
0,38101,Social
1,57308,Referral
2,11865,Paid Search
3,211993,Organic Search
4,3067,Display
5,75688,Direct
6,5966,Affiliates
7,62,(Other)


In [27]:
%%bigquery --project $project_id

SELECT
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY ProductName
ORDER BY ProductName

,ProductName
0,1 oz Hand Sanitizer
1,14oz Ceramic Google Mug
2,15 oz Ceramic Mug
3,"15"" Android Squishable - Online"
4,16 oz. Hot and Cold Tumbler
...,...
628,YouTube Women's Short Sleeve Tri-blend Badge T...
629,YouTube Women's Short Sleeve Tri-blend Badge T...
630,YouTube Womens 3/4 Sleeve Baseball Raglan Whit...
631,YouTube Wool Heather Cap Heather/Black


In [28]:
%%bigquery --project $project_id

SELECT
  COUNT(*) AS product_views,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,ProductName
0,316482,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,22 oz YouTube Bottle Infuser
2,210700,YouTube Men's Short Sleeve Hero Tee Black
3,202205,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,YouTube Custom Decals


In [29]:
%%bigquery --project $project_id

WITH unique_product_views_by_person AS (
-- find each unique product viewed by each visitor
SELECT
 fullVisitorId,
 (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY fullVisitorId, v2ProductName )


-- aggregate the top viewed products and sort them
SELECT
  COUNT(*) AS unique_view_count,
  ProductName
FROM unique_product_views_by_person
GROUP BY ProductName
ORDER BY unique_view_count DESC
LIMIT 5

,unique_view_count,ProductName
0,152358,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,143770,22 oz YouTube Bottle Infuser
2,127904,YouTube Men's Short Sleeve Hero Tee Black
3,122051,YouTube Twill Cap
4,121288,YouTube Custom Decals


In [30]:
%%bigquery --project $project_id

SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,v2ProductName
0,316482,3158,6352,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,22 oz YouTube Bottle Infuser
2,210700,949,1114,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,YouTube Custom Decals


In [31]:
%%bigquery --project $project_id

SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  SUM(productQuantity) / COUNT(productQuantity) AS avg_per_order,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,avg_per_order,ProductName
0,316482,3158,6352,2.011400,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,9.387795,22 oz YouTube Bottle Infuser
2,210700,949,1114,1.173867,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,2.975304,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,6.656489,YouTube Custom Decals


## Data Prediction


**Selecting features**

1- Select features and create training dataset.

we have decided to test whether these two fields are good inputs for my classification model:

totals.bounces (whether the visitor left the website immediately)

totals.timeOnSite (how long the visitor was on our website)

In [32]:
%%bigquery --project $project_id
SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1)
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
ORDER BY time_on_site DESC
LIMIT 10;

,bounces,time_on_site,will_buy_on_return_visit
0,0,15047,0
1,0,12136,0
2,0,11201,0
3,0,10046,0
4,0,9974,0
5,0,9564,0
6,0,9520,0
7,0,9275,1
8,0,9138,0
9,0,8872,0


Question: Which fields are the input features and the label?

Answer: The inputs are bounces and time_on_site. The label is will_buy_on_return_visit.

Question: Which two fields are known after a visitor’s first session?

Answer: bounces and time_on_site are known after a visitor’s first session.

**creating dataset**

**Creating Model:**

In [33]:
#Select features and create your training dataset
%%bigquery --project $project_id
SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1)
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
ORDER BY time_on_site DESC
LIMIT 10;

,bounces,time_on_site,will_buy_on_return_visit
0,0,15047,0
1,0,12136,0
2,0,11201,0
3,0,10046,0
4,0,9974,0
5,0,9564,0
6,0,9520,0
7,0,9275,1
8,0,9138,0
9,0,8872,0


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE MODEL `bqml_ecomm.model`
OPTIONS
(
model_type='logistic_reg',
labels = ['will_buy_on_return_visit']
)
AS

#standardSQL
SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430') # train on first 9 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
;


Executing query with job ID: cb43dbda-e94d-48b6-b824-e8ce566663a9
Query executing: 88.81s

**Evaluating:**

In [ ]:
%%bigquery --project $project_id

SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL `bqml_ecomm.model`,  (

SELECT
  * EXCEPT(fullVisitorId)
FROM

  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20170430' and '20170630') # train on first 9 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId);)

)
)





**Recreating model after adding more features:**

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE MODEL `bqml_ecomm.model`
OPTIONS
  (model_type='logistic_reg', labels = ['will_buy_on_return_visit']) AS

WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)

# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,

      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430' # train 9 months

  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
);


**Evaluating:**

In [ ]:
%%bigquery --project $project_id

SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL `bqml_ecomm.model`,  (

WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)

# add in new features
SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,

      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20170501' AND '20170630' # eval 2 months

  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)
));


**Prediction:**

In [ ]:
%%bigquery --project $project_id

SELECT
*
FROM
  ml.PREDICT(MODEL `bqml_ecomm.model`,
   (

WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)

  SELECT
      CONCAT(fullvisitorid, '-',CAST(visitId AS STRING)) AS unique_session_id,

      # labels
      will_buy_on_return_visit,

      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # mobile or desktop
      device.deviceCategory,

      # geographic
      IFNULL(geoNetwork.country, "") AS country

  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE
    # only predict for new visits
    totals.newVisits = 1
    AND date BETWEEN '20170701' AND '20170801' # test 1 month

  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)

)

ORDER BY
  predicted_will_buy_on_return_visit DESC;


## Conclusion

---

*TODO:
 Final conclusions based on the rest of your project*
Of the top 6% of first-time visitors (sorted in decreasing order of predicted

probability), more than 6% make a purchase in a later visit.

These users represent nearly 50% of all first-time visitors who make a purchase

in a later visit.

Overall, only 0.7% of first-time visitors make a purchase in a later visit.

Targeting the top 6% of first-time increases marketing ROI by 9x vs targeting 

them all!
---